In [4]:
directory = './EmoDB_dataset/silenceRemoved'

# MFCC features

# to do: global normalisation

## Feature extraction

In [5]:
import os
import librosa
import numpy as np
from typing import Dict

def extract_mfcc_features(file_path: str, n_mfcc: int = 39, 
                          frame_size: float = 0.025, frame_stride: float = 0.01, 
                          n_segments: int = 10) -> np.ndarray:
    """
    Extracts 39 MFCC features framewise from an audio file and then applies
    average pooling to condense the features over time into an n x 39 feature matrix.
    
    Parameters:
      file_path (str): Path to the audio file.
      n_mfcc (int): Number of MFCC features to extract. Default is 39.
      frame_size (float): Length of each frame in seconds. Default is 0.025.
      frame_stride (float): Step between successive frames in seconds. Default is 0.01.
      n_segments (int): Number of segments (n) to pool the frames into.
    
    Returns:
      np.ndarray: A n x 39 array where each row is the average MFCC vector for that segment.
    """
    try:
        signal, sample_rate = librosa.load(file_path, sr=None)
        frame_length = int(frame_size * sample_rate)
        hop_length = int(frame_stride * sample_rate)
        
        # Extract MFCC features; result shape is (n_mfcc, T) where T is number of frames.
        mfcc = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=n_mfcc,
                                    n_fft=frame_length, hop_length=hop_length)
        
        # Normalize the MFCC features along each coefficient dimension.
        mfcc_normalized = mfcc - np.mean(mfcc, axis=1, keepdims=True)
        
        # Transpose to shape (T, n_mfcc) for pooling along the time axis.
        mfcc_normalized = mfcc_normalized.T
        
        # Divide the frames into n_segments segments and compute the average for each segment.
        segments = np.array_split(mfcc_normalized, n_segments, axis=0)
        pooled_features = np.array([np.mean(seg, axis=0) for seg in segments])
        
        return pooled_features  # Shape: (n_segments, 39)
    
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    return np.array([])

def process_directory_mfcc(directory: str, n_segments: int = 10) -> Dict[str, np.ndarray]:
    """
    Processes all .wav files in the given directory, extracting their MFCC features
    using average pooling to produce an n x 39 feature matrix for each file.
    
    Parameters:
      directory (str): Path to the directory containing .wav files.
      n_segments (int): Number of segments to pool the frames into for each file.
    
    Returns:
      Dict[str, np.ndarray]: A dictionary mapping filenames to their corresponding feature matrices.
    """
    feature_vectors = {}
    for filename in os.listdir(directory):
        if filename.endswith('.wav'):
            file_path = os.path.join(directory, filename)
            features = extract_mfcc_features(file_path, n_segments=n_segments)
            if features.size > 0:
                feature_vectors[filename] = features
    return feature_vectors


## Classifier on MFCC

In [6]:
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


def load_labels(csv_file: str) -> pd.DataFrame:
    return pd.read_csv(csv_file)

def prepare_dataset(features: dict, labels: pd.DataFrame):
    """
    Constructs the dataset by matching each audio file's feature matrix with its label.
    Since each file is represented as an n x 39 matrix (n segments by 39 features),
    we flatten it into a 1D feature vector of length n*39.
    """
    X = []
    y = []
    for _, row in labels.iterrows():
        file_id = row['Filename']
        if file_id in features:
            # Flatten the (n, 39) matrix to a 1D vector (n*39,)
            feature_matrix = features[file_id]
            feature_vector = feature_matrix.flatten()
            X.append(feature_vector)
            y.append(int(row['EmotionNumeric']))
    return np.array(X), np.array(y)


def train_and_evaluate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # GMM Classifier
    gmm_model = GaussianMixture(n_components=len(np.unique(y)), random_state=42)
    gmm_model.fit(X_train)
    gmm_predictions = gmm_model.predict(X_test)

    gmm_acc = accuracy_score(y_test, gmm_predictions)
    print("GMM Classifier Report:")
    print(classification_report(y_test, gmm_predictions))

    # SVM Classifier
    svm_model = SVC(kernel='linear', random_state=42)
    svm_model.fit(X_train, y_train)
    svm_predictions = svm_model.predict(X_test)

    svm_acc = accuracy_score(y_test, svm_predictions)
    print("SVM Classifier Report:")
    print(classification_report(y_test, svm_predictions))

    return gmm_acc, svm_acc


# Wandb

In [ ]:
import wandb
import numpy as np

def single_run_all_nmfccs():
    wandb.init(
        project="ssp-mfcc-tuning-SilenceRemoval",
        name="n_mfcc_sweep_single_run",
        config={}
    )

    for n in np.arange(10, 100, 5):
        mfccFeatures = process_directory_mfcc(directory, n)
        labels = load_labels("EmoDB_dataset/emotion_mapping_detailed.csv")
        X, y = prepare_dataset(mfccFeatures, labels)
        gmm_acc, svm_acc = train_and_evaluate(X, y)

        # Log scalars at each step
        wandb.log({
            "n_mfcc": n,
            "svm_accuracy": svm_acc,
            "gmm_accuracy": gmm_acc,
            "feature_dim": X.shape[1]
        })

    wandb.finish()

# Run it directly
single_run_all_nmfccs()


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


GMM Classifier Report:
              precision    recall  f1-score   support

           0       0.33      0.11      0.17        18
           1       0.28      0.35      0.31        20
           2       0.10      0.08      0.09        12
           3       0.11      0.07      0.09        14
           4       0.18      0.33      0.23        18
           5       0.24      0.56      0.33         9
           6       0.50      0.06      0.11        16

    accuracy                           0.21       107
   macro avg       0.25      0.22      0.19       107
weighted avg       0.26      0.21      0.19       107

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.48      0.78      0.60        18
           1       0.60      0.60      0.60        20
           2       0.50      0.25      0.33        12
           3       0.50      0.57      0.53        14
           4       0.69      0.50      0.58        18
           5       0.50      0.7

c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


GMM Classifier Report:
              precision    recall  f1-score   support

           0       0.20      0.11      0.14        18
           1       0.10      0.05      0.07        20
           2       0.15      0.17      0.16        12
           3       0.12      0.21      0.15        14
           4       0.00      0.00      0.00        18
           5       0.20      0.22      0.21         9
           6       0.06      0.12      0.09        16

    accuracy                           0.11       107
   macro avg       0.12      0.13      0.12       107
weighted avg       0.11      0.11      0.11       107

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.38      0.56      0.45        18
           1       0.71      0.50      0.59        20
           2       0.20      0.08      0.12        12
           3       0.39      0.50      0.44        14
           4       0.47      0.44      0.46        18
           5       0.46      0.6

...

GMM Classifier Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.10      0.05      0.07        20
           2       0.06      0.08      0.07        12
           3       0.00      0.00      0.00        14
           4       0.13      0.39      0.19        18
           5       0.00      0.00      0.00         9
           6       0.00      0.00      0.00        16

    accuracy                           0.08       107
   macro avg       0.04      0.07      0.05       107
weighted avg       0.05      0.08      0.05       107



c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.44      0.61      0.51        18
           1       0.55      0.55      0.55        20
           2       0.29      0.17      0.21        12
           3       0.50      0.36      0.42        14
           4       0.46      0.33      0.39        18
           5       0.39      0.78      0.52         9
           6       0.50      0.44      0.47        16

    accuracy                           0.46       107
   macro avg       0.45      0.46      0.44       107
weighted avg       0.46      0.46      0.45       107



c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


ValueError: Input X contains NaN.
GaussianMixture does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values